In [39]:
from census import Census
import us
import pandas as pd

In [ ]:
apikey = ""
fields = ['NAME', 'B01001_007E', 'B01001_008E']

In [51]:
def create_year_pop_df(ls,year):
    df = pd.DataFrame(ls)
    df['Population Over 65'] = df['B01001_007E']+df['B01001_008E']
    df['Year'] = year
    df.drop(df.index[df['NAME'] == 'District of Columbia'], inplace=True)
    df.drop(df.index[df['NAME'] == 'Puerto Rico'], inplace=True)
    dfother = df[['NAME','state','Population Over 65', 'Year']]
    return dfother

In [54]:
def create_dfs(poy):
    years = [y for y in range(2014,2024)]
    df1 = create_year_pop_df(poy[0],2014)
    for x in range(1,len(years)):
        df2 = create_year_pop_df(poy[x],years[x])
        df1 = pd.concat([df1,df2])
    return(df1)

In [56]:
c = Census(apikey)
abfips = us.states.mapping('fips','abbr')
popsoveryear = []
tracker = 0
for y in range(2014,2024):
    popsoveryear.append(c.acs5.get((fields),
          {'for': 'state:*'},year = y))
    tracker+=1
df8 = create_dfs(popsoveryear)
df8.to_csv('./Data/StatePopulations.csv',header=True,index=False)